# <center><font color="red"> https://bit.ly/ptpjb-2021-18</font>
# <center><font color="blue">18 - Introduction to Streamlit for Machine Learning Deployment</font>

<center><img alt="" src="images/cover_ptpjb_2021.png"/></center> 

## <center><font color="blue">tau-data Indonesia</font><br>(C) Taufik Sutanto - 2021</center>
<center><a href="https://tau-data.id">https://tau-data.id</a> ~ <a href="mailto:taufik@tau-data.id">taufik@tau-data.id</a></center>

<img src="images/meme_deploy.jpeg"/>

<img src="images/alur_ml_deploy.jpeg" width="700"/>

## Why Deploy Machine Learning Models?

- Deployment model machine learning adalah proses untuk membuat model tersedia dalam tahap produksi.
- Sehingga model terbaik yang telah dibuat dapat digunakan dalam bentuk aplikasi web, perangkat lunak perusahaan, atau API dengan menyediakan data observasi baru untuk menghasilkan prediksi.

## Deployment Tools

<img src="images/webframeworks_python.jpg" width="700"/>
<img src="images/heroku_logo.png" width="400"/>
<img src="images/streamlit_logo.png" width="400"/>

image source:
- https://www.activestate.com/blog/the-top-10-python-frameworks-for-web-development/
- https://www.keyzex.com/2018/09/apa-itu-heroku-cara-deploy-di-heroku.html?m=1
- https://ichi.pro/id/streamlit-merevolusi-pembuatan-aplikasi-data-248940993548562

## Streamlit
link: https://streamlit.io/
<img src="images/streamlit.JPG" width="700"/>
- **Streamlit** Streamlit adalah sebuah framework berbasis Python dan bersifat open-source yang dibuat untuk memudahkan dalam membangun apikasi web di bidang data science dan machine learning yang interaktif.
- Salah satu hal menarik dari framework ini adalah kita tidak perlu mengetahui banyak hal tentang teknologi web development. 
- Kita tidak perlu dipusingkan tentang bagaiamana mengatur tampilan website dengan CSS, HTML, atau Javascript. 
- Untuk menggunakan Streamlit, kita cukup memiliki modal dasar mengetahui bahasa Python saja.

## Contoh aplikasi web yang dibuat di streamlit
- **Sistem Deteksi Objek Real-Time:** Aplikasi ini dibangun dengan menggunakan dataset Self Driving Car Udacity. Aplikasi tersebut mampu mendeteksi objek secara real-time, menggunakan algoritma deteksi objek YOLO di backend yang terjalin dengan Streamlit di frontend.

<img src="images/streamlit_app_1.gif"/>

- **Face GAN Explorer:** Aplikasi ini mampu menghasilkan wajah fotorealistik dan dibangun dengan TensorFlow menggunakan Nvidia's Progressive Growing of GANs dan metode Transparent Latent-space GAN milik Shaobo Guan untuk menyetel karakteristik wajah keluaran.

<img src="images/streamlit_app_2.gif"/>

- **Peramban Data Geografis untuk NYC:** Aplikasi ini menggunakan data penjemputan Uber dari New York City untuk secara interaktif memvisualisasikan penjemputan uber dan perjalanan penumpang melintasi NYC.


<img src="images/streamlit_app_3.gif"/>

image source: https://ichi.pro/id/streamlit-merevolusi-pembuatan-aplikasi-data-248940993548562

## Deploy Anomaly Detection App

kita akan coba bagaimana melakukan deployment sederhana menggunakan streamlit pada model anomaly detection yang telah kita buat. hasil deployment yang akan kita buat akan memiliki tampilan seperti berikut:

<img src="images/streamlit app.JPG" width="700"/>

## Persiapan
**Lakukan instalasi streamlit pada command prompt atau anaconda prompt:**
- `> pip install streamlit`
- `> streamlit hello`

**Menyiapkan file yang diperlukan:**
- model anomaly detection yang telah dibuat
- feature scaling yang digunakan
- data untuk mencoba aplikasi anomaly detection

file-file di atas dapat dipersiapkan pada modul jupyter notebook **14 - Deep Learning ~ LSTM** dan di simpan pada folder streamlit.

**Membuat text-file python**
kemudian pada folder deployment tersebut kita buat text-file dengan format `.py` untuk menuliskan code deployment menggunakan streamlit.

<img src="images/text-file 1.JPG"/>
<img src="images/text-file 2.JPG"/>

## Code in Python

Pada file `LSTM_AD.py` kita tuliskan code seperti di bawah ini. kemudian kita akan bahas kegunaannya baris-perbaris.

In [ ]:
"""The App."""

import pandas as pd
import numpy as np
import streamlit as st
from tensorflow import keras
import pickle
import plotly.graph_objects as go

# Create sliding window
def sliding_window(data, window_size):
    sub_seq, next_values = [], []
    for i in range(len(data)-window_size):
        sub_seq.append(data[i:i+window_size])
        next_values.append(data[i+window_size])
    X = np.stack(sub_seq)
    y = np.array(next_values)
    return X,y

window_size = 30

# Load the model from the file
model = keras.models.load_model('anomaly_detection')

# load the scaler
scaler = pickle.load(open('scaler.pkl', 'rb'))

threshold = 527.8798828125

st.write("""
# LSTM Anomaly Detection App for Web Traffic Data
""")

st.write("""
### Data format and must be greater than 30 timestamps
| timestamp  | value   |
| -----------|:-------:|
| 1          | 10      |
| 2          | 7       |
| 3          | 17      |
""")

uploaded_file = st.file_uploader("Choose a file", type='csv')

if uploaded_file is not None:

    df = pd.read_csv(uploaded_file)
    df['scaled'] = scaler.transform(df[['value']])
    
    X, y = sliding_window(df[['scaled']].values, window_size)
    
    predict = scaler.inverse_transform(model.predict(X))
    y = scaler.inverse_transform(y)
    
    abs_error = np.abs(y - predict)
    
    test_anomaly = pd.DataFrame()
    test_anomaly['timestamp'] = df['timestamp'][window_size:]
    test_anomaly['value'] = df['value'][window_size:]
    test_anomaly['abs_error'] = abs_error
    test_anomaly['anomaly_hat'] = 0
    test_anomaly.loc[test_anomaly['abs_error'] >= threshold, 'anomaly_hat'] = 1
    
    anomalies = test_anomaly.loc[test_anomaly['anomaly_hat'] == 1]

    st.write("Visualize Detected Anomalies from Data")  

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=test_anomaly['timestamp'], y=test_anomaly['value'], name='value'))
    fig.add_trace(go.Scatter(x=anomalies['timestamp'], y=anomalies['value'], mode='markers', name='Anomaly'))
    fig.update_layout(showlegend=True, title='Detected anomalies')
    st.plotly_chart(fig)

**Import modul/library yang diperlukan**

In [ ]:
import pandas as pd
import numpy as np
import streamlit as st
from tensorflow import keras
import pickle
import plotly.graph_objects as go

**Membuat fungsi sliding window serta menentukan window size sesuai dengan model anomaly detection yang digunakan**

In [ ]:
# Create sliding window
def sliding_window(data, window_size):
    sub_seq, next_values = [], []
    for i in range(len(data)-window_size):
        sub_seq.append(data[i:i+window_size])
        next_values.append(data[i+window_size])
    X = np.stack(sub_seq)
    y = np.array(next_values)
    return X,y

window_size = 30

**Memuat model anomaly detection dan feature scaling serta menentukan threshold dari hasil model anomaly detection yang dibuat**

In [ ]:
# Load the model from the file
model = keras.models.load_model('anomaly_detection')

# load the scaler
scaler = pickle.load(open('scaler.pkl', 'rb'))

threshold = 527.8798828125

**Menampilkan header dan contoh format data yang benar**

In [ ]:
st.write("""
# LSTM Anomaly Detection App for Web Traffic Data
""")

st.write("""
### Data format and must be greater than 30 timestamps
| timestamp  | value   |
| -----------|:-------:|
| 1          | 10      |
| 2          | 7       |
| 3          | 17      |
""")

Fungsi st.write() dari modul streamlit salah satu fungsinya adalah untuk menampilkan sesuatu dalam format `markdown`. Agar dapat menulis `markdown` dengan beberapa baris maka kita gunakan tanda kutip tiga `(""" <markdown> """)`

Dari code di atas kita akan peroleh tampilan sebagai berikut:

<img src="images/header.JPG" width="600"/>

**Membuat tombol upload**

In [ ]:
uploaded_file = st.file_uploader("Choose a file", type='csv')

Fungsi st.file_uploader() dari modul streamlit berfungsi untuk membuat tombol upload. Argumen `"Choose a file"` dan `type='csv'` berarti kita menentukan judul pada tombol upload dan tipe file yang dapat diupload.

Dari code di atas kita akan peroleh tampilan sebagai berikut:

<img src="images/file upload.JPG"/>

**Memproses data yang diupload**

Setelah data diupload kita gunakan code `if uploaded_file is not None:` untuk memperoses data tersebut dengan langkah-langkah seperti pada modul jupyter notebook **14 - Deep Learning ~ LSTM**

untuk menampilkan grafik kita gunakan st.plotly_chart() dari modul streamlit dengan argumen `fig` yaitu variabel grafik dari modul plotly

In [ ]:
if uploaded_file is not None:

    df = pd.read_csv(uploaded_file)
    df['scaled'] = scaler.transform(df[['value']])
    
    X, y = sliding_window(df[['scaled']].values, window_size)
    
    predict = scaler.inverse_transform(model.predict(X))
    y = scaler.inverse_transform(y)
    
    abs_error = np.abs(y - predict)
    
    test_anomaly = pd.DataFrame()
    test_anomaly['timestamp'] = df['timestamp'][window_size:]
    test_anomaly['value'] = df['value'][window_size:]
    test_anomaly['abs_error'] = abs_error
    test_anomaly['anomaly_hat'] = 0
    test_anomaly.loc[test_anomaly['abs_error'] >= threshold, 'anomaly_hat'] = 1
    
    anomalies = test_anomaly.loc[test_anomaly['anomaly_hat'] == 1]

    st.write("Visualize Detected Anomalies from Data")  

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=test_anomaly['timestamp'], y=test_anomaly['value'], name='value'))
    fig.add_trace(go.Scatter(x=anomalies['timestamp'], y=anomalies['value'], mode='markers', name='Anomaly'))
    fig.update_layout(showlegend=True, title='Detected anomalies')
    st.plotly_chart(fig)

Dari code di atas kita akan peroleh tampilan sebagai berikut setelah file diupload:<br>
<img src="images/uploaded file.JPG" width="600"/>

## Menjalankan Aplikasi Web Streamlit
Aplikasi web streamlit dapat dijalankan di **lokal** atau bisa  melalui **share streamlit**
- **Menjalankan aplikasi web streamlit di lokal**
1. Buka command prompt atau anaconda prompt dan arahkan ke direktori penyimpanan file aplikasi web streamlit (misal: LSTM_AD.py) berada

<img src="images/streamlit_local12.JPG"/>
<img src="images/streamlit_local1.JPG"/>

2. pada cmd, ketik: `streamlit run LSTM_AD.py`

<img src="images/streamlit_local2.JPG"/>

3. kemudian tekan `Enter` dan streamlit akan membuka aplikasi web melalui browser

<img src="images/streamlit_local3.JPG"/>

- **Menjalankan aplikasi web streamlit melalui share streamlit**

**Persiapan:**
1. Memiliki akun GitHub dan sign up pada laman https://streamlit.io/sharing-sign-up
2. Kemudian hubungkan akun streamlit dengan GitHub (memerlukan 1-2 hari kerja)
3. siapkan repository GitHub yang berisikan file-file seperti saat dijalankan di lokal ditambah text-file bernama requirements.txt yang berisikan library python berserta versi yang digunakan seperti gambar berikut.

<img src="images/requirements_streamlit.JPG"/>

**Langkah-langkah:**
1. setelah persiapan di atas dilakukan, sign in pada laman https://share.streamlit.io/
2. klik New app

<img src="images/streamlit_share1.JPG"/>

3. isikan repository untuk deployment dan main file pathnya adalah nama file aplikasi web streamlit. kemudian klik deploy

<img src="images/streamlit_share2.JPG"/>

4. maka aplikasi web akan diproses dan jika sudah maka hasilnya seperti gambar berikut
<img src="images/streamlit_share3.JPG"/>

## Latihan

<center><h3>Buatlah aplikasi web model regresi dari tabel berikut menggunakan streamlit sehingga aplikasi web yang dihasilkan seperti gambar berikut!</h3><center/>
    
<center><h3>Kemudian deploy aplikasi web tersebut ke share streamlit!</h3><center/>

## Penutup
Masih banyak lagi cara untuk deployment model machine learning menggunakan streamlit. kita dapat mengeksplor lagi dokumentasi streamlit pada link berikut: https://docs.streamlit.io/en/stable/api.html ataupun melihat hasil deployment yang telah dibagikan pada link berikut: https://streamlit.io/gallery. Selain itu dapat juga didesain terlebih dahulu deployment yang ingin dilakukan agar lebih terarah.

# <center><font color="blue">Akhir Modul 18 - Introduction to Streamlit for Machine Learning Deployment
    
<hr />
<img alt="" src="images/meme-cartoon/meme deployment.jpg" />